# Job Nature and Company Profiles (Topic Modelling)

#What are the types of jobs that are in demand in Armenia? How are the job natures changing over time? 

#On the business question of Job Nature and Company Profiles. Unsupervised learning techniques, such as topic modelling and other techniques such as term frequency counting will be applied to the data, including time period segmented dataset. Qualitative assessment will be done on the results to help us understand the job postings. 

In [ ]:
import os
import json
import string
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import re
import logging
import gensim
from gensim import corpora
import spacy

In [ ]:
df_ori = pd.read_csv('C:\\Users\\deeparam\\Desktop\\DMCA\\Sols_DSP_Capstone_Projects_PS_Chegg_2019\\6. TEXT MINING - ONLINE JOB POSTING\data job posts.csv')

In [ ]:
df_ori.head()
print(df_ori.shape)
df = df_ori.drop_duplicates(['jobpost', 'Title'])
print(df.shape)
print("Removed {0} duplicates (based on jobpost + Title)".format(df_ori.shape[0] - df.shape[0]))

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
def preprocess(tokens):
    tokens_nop = [t for t in tokens if t not in string.punctuation]
    tokens_nop = [t.lower() for t in tokens_nop]
    # wnl = nltk.WordNetLemmatizer()
    stop = stopwords.words('english')
    stop.extend(
        ['armenian', 'armenia', 'job', 'title', 'position', 'location', 'responsibilities', 'application', 'procedures',
         'deadline', 'required', 'qualifications', 'renumeration', 'salary', 'date', 'company', 'yerevan',
         'eligibility', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september',
         'october', 'november', 'december'])
    tokens_nostop = [t for t in tokens_nop if t not in stop]
    # tokens_lem = [wnl.lemmatize(t) for t in tokens_nostop]
    tokens_clean = [t for t in tokens_nostop if len(t) >= 3]  # simple way to remove the offending " punctuations
    return tokens_clean


In [ ]:
def plotWC(tokens):
    text_clean = " ".join(tokens)
    print(text_clean)
    wc = WordCloud(background_color="white").generate(text_clean)
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    plt.figure(figsize=(20, 9))
    fd = nltk.FreqDist(tokens)  # case
    fd.plot(50)

In [ ]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [ ]:
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [ ]:
import spacy
#import en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [ ]:
#lemmatization

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
  
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
df['jobpost'] = df['jobpost'].apply(lambda x: re.sub('\S*@\S*\s?', '', x))  # remove emails
df['jobpost'] = df['jobpost'].apply(lambda x: re.sub('\s+', ' ', x))  # remove newlines
df['jobpost'] = df['jobpost'].apply(lambda x: re.sub("\'", "", x))  # remove single quotes
re1 = '(www)'  # Word 1
re2 = '(\\.)'  # Any Single Character 1
re3 = '((?:[a-z][a-z0-9_]*))'  # Variable Name 1
re4 = '(\\.)'  # Any Single Character 2
re5 = '((?:[a-z][a-z0-9_]*))'  # Variable Name 2
rg = re.compile(re1 + re2 + re3 + re4 + re5, re.IGNORECASE | re.DOTALL)
df['jobpost'] = df['jobpost'].apply(lambda x: re.sub(rg, "", x))
re1 = '((?:[a-z][a-z0-9_]*))'  # Variable Name 1
re2 = '(\\.)'  # Any Single Character 1
re3 = '((?:[a-z][a-z0-9_]*))'  # Word 1
rg = re.compile(re1 + re2 + re3, re.IGNORECASE | re.DOTALL)
df['jobpost'] = df['jobpost'].apply(lambda x: re.sub(rg, "", x))
df.jobpost = df.jobpost.apply(lambda x: re.sub('(\\d+)', "", x))  # remove numbers

df['jobpost_token'] = df.jobpost.map(word_tokenize)
df['jobpost_len'] = df.jobpost_token.apply(len)
df['jobpost_token_uniq'] = df.jobpost_token.apply(set)
df['jobpost_processed'] = df.jobpost_token.apply(preprocess)


In [ ]:
# Build the bigram and trigram models

bigram = gensim.models.Phrases(df['jobpost_processed'], min_count=5, threshold=100) # higher threshold fewer phrases.

In [ ]:
trigram = gensim.models.Phrases(bigram['jobpost_processed'], threshold=100)

In [ ]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
df.jobpost_processed = make_bigrams(df.jobpost_processed)
df.jobpost_processed = lemmatization(df.jobpost_processed, allowed_postags=['NOUN', 'VERB'])  # 'ADJ',, 'ADV'])

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
dictionary = corpora.Dictionary(df['jobpost_processed'])
#print(dictionary)
dictionary.filter_extremes(no_below=3, no_above=0.7)
#print(dictionary)

In [ ]:

topic_num = 7

# Use the dictionary to prepare a DTM (using TF)
dtm_train = [dictionary.doc2bow(d) for d in df['jobpost_processed']]
lda = gensim.models.ldamodel.LdaModel(dtm_train, num_topics=topic_num, alpha='auto', chunksize=30, id2word=dictionary,
                                      passes=20, random_state=432)
# lda.show_topics()
# lda.show_topics(num_words=20)

dtopics_train = lda.get_document_topics(dtm_train)
# print topic distribution for 1st 5 rows
for i in range(0, 5):
    print(dtopics_train[i])




In [ ]:
print(lda.print_topics(num_topics=2, num_words=4))

In [ ]:
for i in lda.print_topics(): 
    for j in i: print (j)

In [ ]:
lda.show_topics()


7 topics

0, Sales and Marketing 

1, Project Management and Development 

2, Banking and Finance

3, Software Development 

4, Construction and Safety Engineering 

5, Education and Training 

6, Business Development and Management 



In [ ]:
# get and plot the distribution of the topics
from operator import itemgetter
import matplotlib.style as style

import seaborn as sns

# style.use('seaborn-poster')
# fig, ax = plt.subplots(figsize=(16, 9))
top_train = [max(t, key=itemgetter(1))[0] for t in dtopics_train]
# plt.hist(top_train, bins=topic_num)
# plt.title('Topic Frequencies')
# plt.show()
sns.distplot(top_train, kde=False).set_title('Topic Frequencies')

In [ ]:
topics = pd.Series(top_train)
df['topic'] = topics.values
df = pd.DataFrame(df)

In [ ]:
df_04to07 = df[df.Year <= 2007]
sns.distplot(df_04to07['topic'], kde=False).set_title('Topic Frequencies 2004 - 2007')

In [ ]:
df_08to11 = df[(df['Year'] > 2007) & (df['Year'] <= 2011)]
sns.distplot(df_08to11['topic'], kde=False).set_title('Topic Frequencies 2008 - 2011')

In [ ]:
df_12to15 = df[(df['Year'] > 2011) & (df['Year'] <= 2015)]
sns.distplot(df_12to15['topic'], kde=False).set_title('Topic Frequencies 2012 - 2015')

In [ ]:
#Conclusion:

#As can be observed, proportion-wise, the topic frequencies remain roughly similar. What is obvious is that the two topics Banking and Finance as well as Software Development have grown in proportion across the overall job market over the years. Also, the highest number of jobs is for a Sales and Marketing position. Construction and Safety Engineering kinds of job consistently had low numbers of job postings over the years.

#There were slightly less Software Development jobs compared to Banking and Finance jobs initially from 2004-2007. This trend continued from 2008 to 2011. However, from 2012-2015, Software Development jobs were higher in demand than Banking and Finance jobs.

A possible explanation could be that banks and finance services have become more digitalised, and many jobs for the Banking and Finance industry require Software Development skills as well.  
 